In [1]:
import polars as pl

In [3]:
names_df = pl.read_csv('data/Student_Name.csv')
nationality_df = pl.read_csv('data/Student_Nationality.csv')

In [4]:
names_df.head()

Student ID,Name
i64,str
1,"""John Smith"""
2,"""Emily Wong"""
3,"""Miguel Diaz"""
4,"""Sarah Johnson"""
5,"""Wei Chen"""


In [7]:
nationality_df.head()

Student ID,Nationality,Classroom
i64,str,str
1,"""USA""","""A"""
2,"""Canada""","""A"""
3,"""Mexico""","""A"""
4,"""USA""","""A"""
5,"""China""","""A"""


In [23]:
df = (
    nationality_df
    .join(names_df, on='Student ID', how='left')
    .group_by('Nationality', 'Classroom')
    .agg(
        pl.count().alias('Student Cnt')
    )
    .with_columns(
        pl.col('Student Cnt').rank('ordinal', descending=True).over('Classroom').alias('Rank')
    )
    .filter(pl.col('Rank')==1)
    .select(pl.exclude('Rank'))
)
df

Nationality,Classroom,Student Cnt
str,str,u32
"""USA""","""A""",5
"""Brasil""","""B""",6
